# Generative AI

![AI_trend](img/AI_trend.jpg)

Large Language Model (LLM) is a foundational model to serve tasks that concern language.

![masked_model](img/masked-model.png)

Training of foundational models is done in semi-supervised way. The same sentence serves as input and output, however on the input side some tokens are masked. When run on large amounts of data, the models are able to capture the semantics and structure of a language and can perform well over numerous tasks.

### Set up the LLM

In [1]:
# import environment variables
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
import os
from genai.credentials import Credentials
from genai.model import Model
from genai.schemas import GenerateParams


apikey = os.environ["APIKEY"]
url = os.environ["URL"]
creds = Credentials(apikey, url)

params = GenerateParams(
            decoding_method="greedy",
            repetition_penalty=1.5,
            moderations={"hap": {"input": True, "threshold": 0.75, "output": True}},
        )

llm = Model("google/flan-ul2", params=params, credentials=creds)

In [14]:
response = llm.generate(["Hello, how are you?"])
print(response[0].generated_text)

Good, thanks.


### Introduce Langchain

The idea of composing components together in a chain is simple but powerful. It drastically simplifies and makes more modular the implementation of complex applications, which in turn makes it much easier to debug, maintain, and improve your applications.
Read more on chains with langchain: https://python.langchain.com/docs/modules/chains/

In [4]:
from genai.extensions.langchain import LangChainInterface
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm_langchain = LangChainInterface(model="google/flan-ul2", params=params, credentials=creds)

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}",
)
llm_chain = LLMChain(llm=llm_langchain, prompt=prompt)

llm_chain.run("Hello, how are you?")

'Good, thanks.'

### Prompt engineering

Quality of the LLM response will depend on what prompt you use to receive answer. Prompts can be as simple as instruction or question that you pass to your model. More advanced prompts can contain also additional context or examples for the model to learn from.
Good reading on prompt enginnering can be found here: https://www.promptingguide.ai/

Example of a single prompt:

Prompt:
```
The sky is
```

In [5]:
llm_chain.run("The sky is")

'blue.'

We can improve the model by giving it also an instruction. Such as:

In [6]:
llm_chain.run("Complete the sentence below.\n The sky is")

'blue because the sun is out and it is a sunny day.'

In [ ]:
# Try out some other instructions to get:
# - response that is poetic/creative
# - short response
# - logic and scientific

In [11]:
# type your code here

'the sky is blue because of refraction of light'

With langchain, you can also reformat your prompt so that you do not have to always include the instruction.

In [16]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="""Complete the sentence below:
    {input}""",
)
llm_chain = LLMChain(llm=llm_langchain, prompt=prompt, verbose=True)

llm_chain.run("The sky is")



> Entering new LLMChain chain...
Prompt after formatting:
Complete the sentence below:
    The sky is

> Finished chain.


'blue because the sun is out and it is a sunny day.'

In [ ]:
# Change the prompt format so that it can accomodate following examples:

# question: What is the capital of Slovakia?
# answer: Bratislava

# question: What is the capital of Brazil?
# answer: Brasilia

In [29]:
# Write your code here

prompt = PromptTemplate(
    input_variables = ["country"], # define your parameter
    template = """
    """
)

llm_chain = LLMChain(llm=llm_langchain, prompt=prompt)
print(llm_chain.run("Slovakia"))
print(llm_chain.run("Brazil"))

# Question: How can you make the LLM to answer with capital letters (e.g. not bratislava but Bratislava)

Bratislava
Brasilia


## Retrieval Augmented Generation (RAG)

![RAG](img/RAG_chart.png)

### Step 1: Prepare your knowledge base

In [30]:
# First, we read our pdf file
from PyPDF2 import PdfReader

pdf = "data/Ian Goodfellow, Yoshua Bengio, Aaron Courville - Deep Learning (2017, MIT).pdf"
pdf_reader = PdfReader(pdf)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [28]:
print(f"The document has approx. {len(text.split())} words")

The document has approx. 79584 words


### Split the text

In [35]:
# We do not want to treat the document as one big chunk of text, rather, it would be easier to split it into smaller pieces that are retrieved based on their relevancy to our question.

from langchain.text_splitter import CharacterTextSplitter

example_doc = """
Deep learning is part of a broader family of machine learning methods, which is based on artificial neural networks with representation learning. 
The adjective "deep" in deep learning refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems.
ANNs have various differences from biological brains. Specifically, artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog.
Within a nervous system, a neuron, neurone, or nerve cell is an electrically excitable cell that fires electric signals called action potentials across a neural network. 
Neurons communicate with other cells via synapses - specialized connections that commonly use minute amounts of chemical neurotransmitters to pass the electric signal from the presynaptic neuron to the target cell through the synaptic gap. 
"""

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len
)

example_chunks = text_splitter.split_text(example_doc)

# check how the text splitter works
print("chunk 0")
print(example_chunks[0])
print("chunk 1")
print(example_chunks[1])

Created a chunk of size 524, which is longer than the specified 500


chunk 0
Deep learning is part of a broader family of machine learning methods, which is based on artificial neural networks with representation learning. 
The adjective "deep" in deep learning refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.
chunk 1
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.


In [36]:
# exercise:
# write code that will split the text of our pdf document into smaller chunks
# try using RecursiveCharacterTextSplitter instead of CharacterTextSplitter

# Type your code here:

# text_splitter = ...

chunks = text_splitter.split_text(text)

In [37]:
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 3360


### Add embeddings

Embeddings are vector representations of tokens (or words) in a space that keep semantic similarity of words. Vectors that are close together are considered similar. In order to do efective search over our knowledgebase, we need to store our texts as embeddings so that we can easily retrieve most semantically accurate chunks to our query.

In [38]:
# langchain class for embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-mpnet-base-v2",
                model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': False}
                )

# try it out

example_embeddings = embedding_model.embed_documents(example_chunks)
print("Dimension of our embeddings")
print(len(example_embeddings), len(example_embeddings[0]))

/Users/natasa/Projects/brainstorm-2023/workshop/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dimension of our embeddings
4 768


In [41]:
# check with cosine similarity which documents are closer to my example query
from sklearn.metrics.pairwise import cosine_similarity

example_query = "I like dogs and cats"
query_emb = embedding_model.embed_query(example_query)

for i, ex in enumerate(example_embeddings):
    print(example_chunks[i])
    print(cosine_similarity([query_emb], [ex]))

Deep learning is part of a broader family of machine learning methods, which is based on artificial neural networks with representation learning. 
The adjective "deep" in deep learning refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.
[[0.02370083]]
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
[[0.12612669]]
Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems.
AN

### Define vector store

Vector store is a type of database that can store unstructured data in an organized way. Typically, text is stored alongside with its embeddings (or vectors) and search is done as a similarity search over these vectors.
&nbsp;

![vectorstore](img/vectorstore.png)
&nbsp;
source: https://python.langchain.com/docs/modules/data_connection/vectorstores/


In [43]:
from langchain.vectorstores import FAISS
# we will use FAISS for this exercise, but you can easily use Chromadb, Milvus, Elasticsearch, etc.
# https://python.langchain.com/docs/integrations/vectorstores

knowledge_base = FAISS.from_texts(chunks, embedding_model)

In [45]:
# test out your knowledge base

query = "What is deep neural network?"
docs = knowledge_base.similarity_search(query)

In [46]:
docs[0]

Document(page_content='together.Forexample,wemighthavethreefunctions f( 1 ), f( 2 ),and f( 3 )connected\ninachain,toform f(x) = f( 3 )( f( 2 )( f( 1 )(x))).Thesechainstructuresarethemost\ncommonlyusedstructuresofneuralnetworks.Inthiscase, f( 1 )iscalledtheﬁrst\nlayerofthenetwork, f( 2 )iscalledthesecondlayer,andsoon.Theoverall\n168CHAPTER6.DEEPFEEDFORWARDNETWORKS\nlengthofthechaingivesthedepthofthemodel.Itisfromthisterminologythat\nthename“deeplearning”arises.Theﬁnallayerofafeedforwardnetworkiscalled\ntheoutputlayer.Duringneuralnetworktraining,wedrive f(x)tomatch f∗(x).\nThetrainingdataprovidesuswithnoisy,approximateexamplesof f∗(x) evaluated\natdiﬀerenttrainingpoints.Eachexamplexisaccompanied byalabel y f≈∗(x).\nThetrainingexamplesspecifydirectlywhattheoutputlayermustdoateachpoint\nx;itmustproduceavaluethatiscloseto y.Thebehavioroftheotherlayersis\nnotdirectlyspeciﬁedbythetrainingdata.\xa0Thelearningalgorithmmustdecide\nhowtousethoselayerstoproducethedesiredoutput,butthetrainingdatado

In [ ]:
import pickle

# Ideally, this would go to some cloud database, for this demo, we are just using a local storage so let's pickle the knowledge base for later use
with open("knowledge_base.pkl", "wb") as file:
    pickle.dump(knowledge_base, file)

### Step 2: Define your QA chain over documents

In case the code did not run, please use this link to download the `knowledge_base.pkl` directly: https://ibm.box.com/s/133tnvy1fsmxtju3z5vo3insblx7zzhz

In [44]:
# run this cell in case something went wrong in the previous chapter
import pickle
with open("knowledge_base.pkl", "rb") as file:
    knowledge_base = pickle.load(file)

In [47]:
# the easiest way is to use pre-defined qa chain from langchain
from langchain.chains.question_answering import load_qa_chain

params = GenerateParams(
            decoding_method="greedy",
            repetition_penalty=1.5,
            moderations={"hap": {"input": True, "threshold": 0.75, "output": True}},
            min_new_tokens=50,
            max_new_tokens=200
        )


query = "What is deep learning?"

docs = knowledge_base.similarity_search(query)
llm_langchain = LangChainInterface(model="google/flan-ul2", params=params, credentials=creds)

chain = load_qa_chain(llm_langchain, chain_type='stuff')
response = chain.run(input_documents=docs, question=query)

print(response)

To summarize, deep learning, the subject of this book, is an approach to AI. Specifically, it is a type of machine learning, a technique that allows computer systems to improve with experience and data. According to the authors of this book, machine learning is the only viable approach to building AI systems that can operate in complicated, real-world environments. Deep learning is a particular r e i nf o r c e m e n t l e ar ni ng.


## Another way how to define your QA chain

In [52]:
# Step 1: define a prompt to handle user's question

qa_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
    Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use a formal language to answer.

    {context}

    Question: {question}
    Helpful Answer:
    """
)

params = GenerateParams(
    decoding_method="sample",
    temperature=0.5,
    repetition_penalty=1.5,
    moderations={"hap": {"input": True, "threshold": 0.75, "output": True}},
    min_new_tokens=50,
    max_new_tokens=200
)
llm_model = LangChainInterface(model="google/flan-ul2", params=params, credentials=creds)

llm_chain = LLMChain(llm=llm_model, prompt=qa_prompt)

In [56]:
# Step 2: define your chain that will handle creation of context

from langchain.chains import StuffDocumentsChain

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

document_variable_name = "context"

stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)

In [57]:
query = "What is deep learning?"

docs = knowledge_base.search(query, search_type="similarity")
stuff_chain.run({"question": query, "input_documents": docs})

'To summarize, deep learning, the subject of this book, is an approach to AI. Specifically, it is a type of machine learning, a technique that allows computer systems to improve with experience and data. According to the authors of this book, machine learning is the only viable approach to building AI systems that can operate in complicated, real-world environments. Deep learning is a particular r e i nf o r c e m e n t l e ar ni ng.'

## Introducing Retrievers

In [88]:
# Instead of performing the search manually, let the chain handle everything
from langchain.chains import RetrievalQA

faiss_retriever = knowledge_base.as_retriever()

chain = RetrievalQA(combine_documents_chain=stuff_chain, retriever=faiss_retriever)
chain.run(query)

'To summarize, deep learning, the subject of this book, is an approach to AI. Specifically, it is a type of machine learning, a technique that allows computer systems to improve with experience and data. According to the authors of this book, machine learning is the only viable approach to building AI systems that can operate in complicated, real-world environments. Deep learning is a particular'

# Agents

![](./img/agent.png)

LLMs suffer from a limitation: they are limited to the knowledge on which they have been trained and the additional knowledge provided as context; as a result, if a useful piece of information is missing the provided knowledge, the model cannot “go around” and try to find it in other sources.

This is the reason why we need to introduce the concept of Agents. Agents can be seen as applications powered by LLMs and integrated with a set of tools like search engines, databases, websites, and so on. Within an agent, the LLM is the reasoning engine that, based on the user input, is able to plan and execute a set of actions that are needed to fulfill the request.

For more details visit the following website: [Langchain Agents](https://python.langchain.com/docs/modules/agents/)